## 1. 模块库的引入

In [1]:
import faiss  # faiss向量库
import numpy as np  # 处理嵌入向量数据，用于faiss向量检索
import os  # 引入操作系统库，后续配置环境变量和获取文件路径
from openai import OpenAI
from langchain_community.document_loaders import *  # 导入各种类型文档加载器类
from langchain.text_splitter import *  # 导入各种文档分块类
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # 不使用分词并行化操作，避免多线程或多进程环境中运行多个模型引发冲突

# 设置Qwen系列具体模型及对应的调用API密钥，从硅基流动大模型服务平台获得
chat_model = os.getenv('chat_model')
embed_model = os.getenv('embed_model')
api_key = os.getenv('api_key')
base_url = os.getenv('base_url')

USER_AGENT environment variable not set, consider setting it to identify your requests.
USER_AGENT environment variable not set, consider setting it to identify your requests.


## 2. 索引流程

### 2.1 加载文档

In [2]:
def load_document(file_path):
    """
    解析各种文档格式的文件，返回文档内容字符串
    :param file_path: 文档文件路径
    :return: 返回文档内容的字符串
    """
    # 定义文档解析加载器字典，根据文档类型选择对应的文档解析加载器类和输入参数
    DOCUMENT_LOADER_MAPPING = {
        ".pdf": (PDFPlumberLoader, {}),
        ".txt": (TextLoader, {"encoding": "utf8"}),
        ".doc": (UnstructuredWordDocumentLoader, {}),
        ".docx": (UnstructuredWordDocumentLoader, {}),
        ".ppt": (UnstructuredPowerPointLoader, {}),
        ".pptx": (UnstructuredPowerPointLoader, {}),
        ".xlsx": (UnstructuredExcelLoader, {}),
        ".csv": (CSVLoader, {}),
        ".md": (UnstructuredMarkdownLoader, {}),
        ".xml": (UnstructuredXMLLoader, {}),
        ".html": (UnstructuredHTMLLoader, {})
    }
    ext = os.path.splitext(file_path)[1]  # 获取文件扩展名，确定文档类型
    loader_tuple = DOCUMENT_LOADER_MAPPING.get(ext)  # 获取文档对应的文档解析加载器类和参数元组

    if loader_tuple:  # 判断文档格式是否在加载器支持范围
        loader_class, loader_args = loader_tuple  # 解包元组，获取文档解析加载器类和参数
        loader = loader_class(file_path, **loader_args)  # 创建文档解析加载器实例，并传入文档文件路径
        documents = loader.load()  # 加载文档
        content = "\n".join([doc.page_content for doc in documents])  # 多页文档内容组合为字符串
        print(f"文档{file_path}的部分内容为: {content[:100]}...")  # 仅用来展示文档内容的前100个字符
        return content  # 返回文档内容的多页拼合字符串
    print(file_path+f". 不支持的文档类型: '{ext}'")  # 若文件格式不支持，输出信息，返回空字符串.
    return ""


### 2.1 获取文本嵌入

In [3]:
def load_embedding_model():
    """
    加载bge-m3模型
    :return: 返回bge-m3模型嵌入的结果
    """
    client = OpenAI(
            api_key=api_key, # 从https://cloud.siliconflow.cn/account/ak获取
            base_url=base_url
        )
    print(f"加载Embedding模型中")

    def get_embedding(text):
       text = text.replace("\n", " ")
       return client.embeddings.create(input = [text], model=embed_model).data[0].embedding

    # from sentence_transformers import SentenceTransformer
    # get_embedding = SentenceTransformer(os.path.abspath('data/bge-large-zh-v1.5'))
    # print(f"bge-small-zh-v1.5模型最大输入长度: {get_embedding.max_seq_length}")

    return get_embedding

### 2.2 建立索引，即把文本嵌入写入FAISS向量数据库

In [4]:
def indexing_process(folder_path, embedding_model, collection):
    """
    索引流程：加载PDF文件，并将其内容分割成小块，计算这些小块的嵌入向量并将其存储在chromadb向量数据库中。
    :param folder_path: 文档文件路径
    :param embedding_model: 预加载的嵌入模型
    :param collection: chroma_db表，存储id，embedding，chunk
    """
    # 初始化空的chunks列表，用于存储所有文档文件的文本块
    all_chunks = []
    all_ids = []

    # 遍历文件夹中的所有文档文件
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename) # 检查是否为文件
        if os.path.isfile(file_path):
            # 解析文档文件，获得文档字符串内容
            document_text = load_document(file_path)
            print(f"文档 {filename} 的总字符数: {len(document_text)}")

            # todo 配置SpacyTextSplitter分割文本块库
            #text_splitter = SpacyTextSplitter(
            #    chunk_size=512, chunk_overlap=128, pipeline="zh_core_web_sm")

            # 配置RecursiveCharacterTextSplitter分割文本块库参数，每个文本块的大小为512字符（非token），相邻文本块之间的重叠128字符（非token）
            # todo 可以更换为CharacterTextSplitter、MarkdownTextSplitter、PythonCodeTextSplitter、LatexTextSplitter、NLTKTextSplitter等
            text_splitter = RecursiveCharacterTextSplitter( chunk_size=512, chunk_overlap=128 )
            # 将文档文本分割成文本块Chunk
            chunks = text_splitter.split_text(document_text)
            print(f"文档 {filename} 分割的文本Chunk数量: {len(chunks)}")
            # 将分割的文本块添加到总chunks列表中
            all_chunks.extend(chunks)
            # 将分割的文本块id添加到总ids列表中
            all_ids.extend([str(uuid.uuid4()) for _ in range(len(chunks))])

    # 文本块转化为嵌入向量列表，用于计算相似度
    embeddings = [embedding_model(chunk) for chunk in all_chunks]
    # embeddings = [embedding_model.encode(chunk, normalize_embeddings=True).tolist() for chunk in all_chunks]

    # collection.add(ids=all_ids, embeddings=embeddings, documents=all_chunks)
    collection.add(ids=list(map(str, range(len(all_chunks)))), embeddings=embeddings, documents=all_chunks)
    print("文本块Chunk转化为嵌入向量完成")
    print("索引过程完成")
    print("*" * 100)

## 3. 检索流程

In [5]:
def retrieval_process(query, collection, embedding_model, top_k=3):
    """
    检索流程：将用户查询Query转化为嵌入向量，并在Chroma索引中检索最相似的前k个文本块。
    :param query: 用户查询语句
    :param collection: chroma_db表，存储了id，embedding，chunk
    :param embedding_model: 预加载的嵌入模型
    :param top_k: 返回最相似的前K个结果
    :return: 返回最相似的文本块及其相似度得分
    """
    # todo 向量检索流程
    # 将查询转化为嵌入向量
    query_embedding = embedding_model(query)
    results = collection.query(query_embeddings=query_embedding, n_results=top_k)

    print(f"查询语句: {query}")
    print(f"向量检索最相似的前{top_k}个文本块:")
    vector_chunks = []
    # 打印检索到的文本块ID、相似度和文本块信息
    for doc_id, doc, score in zip(results['ids'][0], results['documents'][0], results['distances'][0]):
        print(f"文本块ID: {doc_id}")
        print(f"相似度: {score}")
        print(f"文本块信息:\n{doc}\n")
        vector_chunks.append(doc)

    # todo 关键字检索流程
    # 获取向量数据库中所有文本块
    all_docs = collection.get()['documents']
    # 使用jieba分词对所有文本块进行分词
    tokenized_corpus = [list(jieba.cut(doc)) for doc in all_docs]
    # 获取所有分词结果的bm25查询引擎
    bm25 = BM25Okapi(tokenized_corpus)
    # 对查询语句进行分词
    tokenized_query = list(jieba.cut(query))
    # 计算BM25得分
    bm25_scores = bm25.get_scores(tokenized_query)
    # 获取BM25得分最高的top_k个文本块的索引
    bm25_topk_indices = sorted(range(len(bm25_scores)), key=lambda i: bm25_scores[i], reverse=True)[:top_k]
    bm25_chunks = [all_docs[i] for i in bm25_topk_indices]
    print(f"关键字检索最相似的前{top_k}个文本块:")
    for i, chunk in zip(bm25_topk_indices, bm25_chunks):
        print(f"文本块ID: {i}")
        print(f"BM25得分: {bm25_scores[i]}")
        print(f"文本块信息:\n{chunk}\n")

    retrieved_chunks = list(set(vector_chunks + bm25_chunks))

    print("检索过程完成.")
    print("*" * 100)
    return retrieved_chunks

## 4. 生成流程

In [6]:
def generate_process(query, chunks):
    """
    生成流程：调用Qwen大模型云端API，根据查询和文本块生成最终回复。
    :param query: 用户查询语句
    :param chunks: 从检索过程中获得的相关文本块上下文chunks
    :return: 返回生成的响应内容
    """
    # 构建参考文档内容，格式为“参考文档1: \n 参考文档2: \n ...”等
    context = ""
    for i, chunk in enumerate(chunks):
        context += f"参考文档{i+1}: \n{chunk}\n\n"

    # 构建生成模型所需的Prompt，包含用户查询和检索到的上下文
    prompt = f"根据参考文档回答问题：{query}\n\n{context}"
    # print(f"生成模型的Prompt: {prompt}")

    # 准备请求消息，将prompt作为输入
    messages = [{'role': 'user', 'content': prompt}]

    # 调用大模型API实例化大语言模型
    client = OpenAI(
        api_key=api_key, # 从https://cloud.siliconflow.cn/account/ak获取
        base_url=base_url
    )

    try:
        # 使用大模型生成响应
        responses = client.chat.completions.create(
        model=chat_model,
        messages = messages,
        temperature=0.01,
        top_p=0.95,
        stream=False,
        )

        print("生成过程开始:")
        # 流式响应
        # 初始化变量以存储生成的响应内容
        # generated_response = ""

        # # 逐步获取和处理模型的增量输出
        # for response in responses:
        #     response = response.choices[0].delta.content
        #     if response is not None:
        #         generated_response+=response
        #         print(response, end="")

        # 非流式响应
        generated_response = responses.choices[0].message.content
        print(generated_response)

        print("\n生成过程完成.")
        return generated_response
    except Exception as e:
        print(f"大模型生成过程中发生错误: {e}")
        return None

## 5. 执行整个流程

In [7]:
print("RAG过程开始.")
query="下面报告中涉及了哪几个行业的案例以及总结各自面临的挑战？"

RAG过程开始.


### 1. 执行索引过程

In [8]:
#!pip install -U pip chromadb langchain langchain_community sentence-transformers unstructured pdfplumber python-docx python-pptx markdown openpyxl pandas -i https://pypi.tuna.tsinghua.edu.cn/simple

In [9]:
import chromadb # 引入 Chroma 向量数据库
import uuid # 生成唯一ID
import shutil # 文件操作模块，为了避免既往数据的干扰，在每次启动时清空 ChromaDB 存储目录中的文件

In [10]:
embedding_model = load_embedding_model()

# 为了避免既往数据的干扰，在每次启动时清空 ChromaDB 存储目录中的文件
chroma_db_path = os.path.abspath("chroma_db")
if os.path.exists(chroma_db_path):
    shutil.rmtree(chroma_db_path)

# 创建ChromaDB实例和集合collection
client = chromadb.PersistentClient(chroma_db_path)
collection = client.get_or_create_collection(name="documents")

# 索引流程：加载PDF文件，分割文本块，计算嵌入向量，存储在FAISS向量库中（内存）
indexing_process('data', embedding_model, collection)

加载Embedding模型中
文档data\lesson1.pdf的部分内容为: 数字化转型
1. 数字化转型的背景和意义
1.1 背景
在过去的十年里，数字技术的迅猛发展已彻底改变了企业运营的方方面面。互联网、移动技
术、云计算、大数据、物联网（IoT）以及人工智能（AI）等技术...
文档 lesson1.pdf 的总字符数: 9165
文档 lesson1.pdf 分割的文本Chunk数量: 26
文本块Chunk转化为嵌入向量完成
索引过程完成
****************************************************************************************************


### 2. 执行检索过程

In [11]:
#!pip install jieba rank-bm25

In [12]:
import jieba
from rank_bm25 import BM25Okapi
import logging

# 禁用 jieba 的日志输出
jieba.setLogLevel(logging.WARNING)

In [13]:
# 检索流程：将用户查询转化为嵌入向量，检索最相似的文本块
retrieval_chunks = retrieval_process(query, collection, embedding_model)

查询语句: 下面报告中涉及了哪几个行业的案例以及总结各自面临的挑战？
向量检索最相似的前3个文本块:
文本块ID: 23
相似度: 0.91958567390902
文本块信息:
构建基于云计算的智能供应链管理系统，实现供应链的端到端可视化管理。
2.2 案例二：零售业的数字化转型
2.2.1 公司背景
零售业案例讲述了一家全球知名的快时尚服装零售企业，面对电子商务的崛起和消费者购物
行为的快速变化，传统零售模式受到巨大挑战。为保持市场竞争力并满足消费者日益增长的
数字化需求，公司决定实施全面的数字化转型战略。
2.2.2 面临的挑战
在数字化转型之前，零售业案例的公司面临以下挑战：线上线下渠道割裂，导致库存管理不
统一、客户体验不一致，难以提供无缝购物体验；数据利用率低，尽管拥有大量消费者和销
售数据，但缺乏先进的数据分析工具，未能转化为可操作的商业洞察。

文本块ID: 22
相似度: 0.9451326041044928
文本块信息:
2. 案例分析
2.1 案例一：制造业的数字化转型
2.1.1 公司背景
制造业案例介绍了一家成立于20世纪初的德国老牌汽车制造公司，拥有悠久的历史和丰富
的制造经验。面对日益激烈的市场竞争和消费者需求的变化，公司意识到传统制造模式已无
法适应现代市场需求，因而决定实施全面的数字化转型，以保持竞争力。
2.1.2 面临的挑战
在数字化转型之前，制造业案例公司面临多重挑战：生产效率低下，传统制造流程依赖人工，
导致效率低且易出错；供应链复杂，涉及多个国家和地区，信息传递不及时，造成库存管理
困难，甚至存在供应链断裂的风险；客户需求变化快，传统大规模生产方式无法满足市场对
个性化定制产品的需求。
2.1.3 数字化转型解决方案
为了应对制造业上述挑战，公司通过以下步骤进行数字化转型：首先，引入工业4.0技术，
包括物联网（IoT）、人工智能（AI）、大数据分析和机器人自动化，以优化生产线；其次，
构建基于云计算的智能供应链管理系统，实现供应链的端到端可视化管理。
2.2 案例二：零售业的数字化转型
2.2.1 公司背景
零售业案例讲述了一家全球知名的快时尚服装零售企业，面对电子商务的崛起和消费者购物

文本块ID: 24
相似度: 0.9626499204392852
文本块信息:
2.2.3 数字化转型解决方案
为了解决零售业案例

### 3. 生成回答

In [14]:
# 生成流程：调用Qwen大模型生成响应
generate_process(query, retrieval_chunks)

print("RAG过程结束.")

生成过程开始:
根据提供的参考文档，报告中涉及了三个行业的案例，分别是制造业、零售业和金融业。每个行业面临的挑战如下：

1. **制造业**：
   - 生产效率低下，传统制造流程依赖人工，导致效率低且易出错。
   - 供应链复杂，涉及多个国家和地区，信息传递不及时，造成库存管理困难，甚至存在供应链断裂的风险。
   - 客户需求变化快，传统大规模生产方式无法满足市场对个性化定制产品的需求。

2. **零售业**：
   - 线上线下渠道割裂，导致库存管理不统一、客户体验不一致，难以提供无缝购物体验。
   - 数据利用率低，尽管拥有大量消费者和销售数据，但缺乏先进的数据分析工具，未能转化为可操作的商业洞察。

3. **金融业**：
   - 客户服务模式过时，主要依赖实体网点，导致服务效率低、客户体验差。
   - 金融科技企业带来巨大竞争压力，凭借创新技术和便捷服务吸引大量客户，尤其是年轻一代。
   - 数据孤岛和风险管理滞后，各业务部门缺乏数据共享机制，导致信息无法整合，风险管理效率低。

生成过程完成.
RAG过程结束.
